In [1]:
import json
import os
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from docx import Document
from docx.shared import Pt
import pickle
import os
from tqdm import tqdm
import time
from langdetect import detect
import matplotlib.pyplot as plt
import collections
from IPython.display import display

import spacy.cli
from spacy.language import Language
import math

from IPython.display import display

In [2]:
os.getcwd()

'/Users/eleonoracocciu/Desktop/TESI_magistrale/TEDx/notebook'

In [3]:
path_dataset = "../dataset/"

In [4]:
#spacy.cli.download("it_core_news_lg")

In [5]:
nlp = spacy.load('it_core_news_lg')
nlp.max_length = 1500000

### Functions

In [6]:
def get_statistics(set_of_sentences, name_of_sets):
    total = 0
    for s,n in zip(set_of_sentences, name_of_sets):
        total += len(s)
        print("{0:17}: {1} sentences".format(n, len(s)))
       
    print()
    print("{0:17}: {1} sentences". format("Totale", total))

In [7]:
def handle_brackets(sentence, bracket):
    
    #print(i, sentence)
    #print()
    words_in_brackets = []
    list_idx = []
        
    if bracket == 'square':
        
        #creo la lista di tutte le parole fra parentesi
        words_in_brackets = re.findall(r'\[.*?\]', sentence)
        
        #creo la lista di indici delle parole fra parentesi (con un solo elemento)
        list_idx = [i for i, el in enumerate(sentence.split()) if '[' in el and ']' in el]
        
    elif bracket == 'round':
        
        #creo la lista di tutte le parole fra parentesi
        words_in_brackets = re.findall(r'\(.*?\)', sentence)
        
        #creo la lista di indici delle parole fra parentesi (con un solo elemento)
        list_idx = [i for i, el in enumerate(sentence.split()) if '(' in el and ')' in el]
        
    #creo la lista delle parole che sono fra parentesi ma senza parentesi
    words_not_in_brackets = [w.replace(w[0], '').replace(w[-1], '') for w in words_in_brackets]
    #print("1° ", words_in_brackets)
    
    
    #elementi da eliminare da words_in_brackets
    to_del = []
    
    for i, el1 in enumerate(words_in_brackets):
        
        #se fra parentesi appare uno dei termini nella lista elimino direttamente l'elemento fra parentesi
        for element in ['Suono', 'suono', 'Suoni', 'suoni', 'Applauso', 'Applausi', 'applauso', 'applausi', \
                        'Musica', 'musica', 'voci', 'Voci', 'Risata', 'risata', 'Risate', 'risate', 'Video', \
                       'video', 'Parla', 'parla']:
            if element in el1:
                sentence = sentence.replace(el1, "")
                to_del.append(el1)
            
        #se l'elemento fra parentesi ha due parole o più, elimino le parentesi, ovvero sostituisco l'elemento
        #con il suo corrispondente senza parentesi
        if len(el1.split()) >=2:
            sentence = sentence.replace(el1, words_not_in_brackets[i])
            to_del.append(el1) 

    #elimino gli elementi che avevano più parole all'interno delle parentesi da words_in_brackets
    #e ricalcolo di conseguenza words_not_in_brackets
    words_in_brackets = [el for el in words_in_brackets if el not in to_del]
    #print("2° ", words_in_brackets)
    words_not_in_brackets = [parola.replace(parola[0], '').replace(parola[-1], '') for parola in words_in_brackets]

    #print("Frase in fase parziale", sentence)
    #print()
    #calcolo nuovamente gli indici delle parole fra parentesi (con un solo elemento) perchè potrei aver modificato la frase
    if bracket == 'square':
        list_idx = [i for i, el in enumerate(sentence.split()) if '[' in el and ']' in el]
    elif bracket == 'round':
        list_idx = [i for i, el in enumerate(sentence.split()) if '(' in el and ')' in el]

    #pulisco completamente la frase per processarla con spacy
    for el1, el2 in zip(words_in_brackets, words_not_in_brackets):
        sentence = sentence.replace(el1, el2)
        
    #print("frase completamente ripulita momentaneamente", sentence)
    #print()

    #elaborazione con spacy
    doc = nlp(sentence)
    
    #creo una lista di tuple contenenti, per ogni parola, il testo e la relativa pos
    list_of_tuples = []
    for w in doc:
        list_of_tuples.append((w.text, w.pos_))
        
    #lascio nella lista di tuple soltanto gli elementi che erano fra parentesi nella frase
    list_of_tuples = [i for i in list_of_tuples if i[0] in words_not_in_brackets]

    #creo un dizionario in cui associo all'indice della parola fra parentesi la relativa pos
    dict_idx_pos = {}
    for idx, el in zip(list_idx, list_of_tuples):
        dict_idx_pos[idx] = el[1]

    sentence = sentence.split()
    
    #print("Frase", sentence)
    #print(list_idx)
    #print("Dict_idx_POS", dict_idx_pos)
    

    #MI OCCUPO SOLO DELL'ELEMENTO IN PRIMA POSIZIONE
    #se ho un elemento fra parentesi in prima posizione, lo elimino dalla frase solo se la sua pos non corrisponde 
    #a nessuna di quelle all'interno della lista
    #siccome all'interno della frase può esserci qualsiasi cosa, non elimino parole che erano tra parentesi in 
    #posizioni diverse dalla prima, per evitare che la frase possa perdere significato
    if 0 in dict_idx_pos:
        if dict_idx_pos[0] not in ['DET', 'AUX', 'PRON', 'VERB', 'NOUN', 'ADV', 'SCONJ', 'CCONJ']:
            #print()
            #print("C'è l'elemento?", sentence)
            del sentence[0]
    
    #print("Qui", sentence)
    #print("Elemento 0", sentence[0])
    #gestisco il caso in cui eventuali eliminazioni abbiano lasciato in prima posizione un segno di punteggiatura
    if sentence:
        if sentence[0] in ['.', ',', ';', ':', '!', '?']:
            del sentence[0]
    
    #controllo se sono presenti altri elementi da eliminare
    #elements_to_delete = ['applauso', 'applausi', 'grazie', 'musica', 'risate', 'risata']
    #for element in sentence:
    #    if element.lower() in elements_to_delete:
    #        sentence.remove(element)
            
        
    #ricompongo la frase ed elimino eventuali spazi all'inizio e alla fine
    sentence = " ".join(sentence)
    sentence = sentence.strip()
    
    #print("words_in_brackets", words_in_brackets)
    #print("words_not_in_brackets", words_not_in_brackets)
    #print()
    #print("Frase con " + bracket + " ripulita: ", sentence)
    #print()
    
    return sentence

In [8]:
def handle_special_symbols(sentence):
    
    sentence = sentence.encode('utf-8').split()

    list_emoji_to_del = []
    
    for i, el in enumerate(sentence):
        if el.startswith(b"\xe2"):
            list_emoji_to_del.append(el)

    for symbol in list_emoji_to_del:
        if symbol in sentence:
            sentence.remove(symbol)

    sentence = b" ".join(sentence).decode('utf-8')
    
    return sentence

In [9]:
def clean_sentences(list_of_sentences):
    
    dataset_new = []
    
    for num_sent, sentence in enumerate(tqdm(list_of_sentences)):
        
        #se una frase ha un solo elemento continuo senza neanche processarla e non la inserisco
        if len(sentence.split()) == 1:
            continue
            
        else:

            #-------------------------------------Pulizia della punteggiatura----------------------------------

            #sostituisco le virgolette particolari con gli apici normali
            sentence = sentence.replace('“', "'")
            sentence = sentence.replace('”', "'")
            sentence = sentence.replace('‘', "'")
            sentence = sentence.replace('’', "'")
            sentence = sentence.replace('€', 'euro ')
            
            
            #rimuovo eventuali segni di interpunzione duplicati
            sentence = re.sub('([-,;:!?]){2,}', r'\1', sentence)

            #se la punteggiatura non è separata a destra da uno spazio lo aggiungo
            sentence = re.sub('([,;:!?])', r'\1 ', sentence)
            
            #se un trattino non è separato da spazi li aggiungo
            sentence = re.sub('(-)', r' \1 ', sentence)

            #elimino eventuali spazi supplementari all'inizio e alla fine della frase
            sentence = sentence.strip()
            
            #-------------------------------------Pulizia delle parentesi-------------------------------------
            
            #gestisco eventuali parentesi quadre e tonde
            if '[' and ']' in sentence:
                sentence = handle_brackets(sentence, 'square')
            if '(' and ')' in sentence:
                sentence = handle_brackets(sentence, 'round')
            
            #gestisco eventuali parentesi quadre e tonde singole che erano così già nel testo originale
            sentence = sentence.replace("[", "")
            sentence = sentence.replace("]", "")
            sentence = sentence.replace("(", "")
            sentence = sentence.replace(")", "")
            
            #-------------------------------------Pulizia finale----------------------------------------------
            
            #rimozione di eventuali simboli speciali
            sentence = handle_special_symbols(sentence)
                        
            #rimozione di spazi supplementari
            sentence = re.sub('\s{2,}', " ", sentence)

            #elimino eventuali tag html
            pattern_html = re.compile('<.*?>')
            sentence = re.sub(pattern_html, "", sentence)
            
            #se alcuni segni di interpunzione sono separati a sinistra da uno spazio, elimino lo spazio
            sentence = re.sub('\s([.,;:!?])', r'\1', sentence)
                    
            #elimino intestazioni della frase come A: oppure A.R.: oppure A.R: oppure AR:
            pattern = re.compile('[A-Z]{1}.[A-Z]{1}.?:')
            sentence = re.sub(pattern, "", sentence)
            
            pattern = re.compile('[A-Z]{2}:')
            sentence = re.sub(pattern, "", sentence)
            
            pattern = re.compile('[A-Z]{1}:')
            sentence = re.sub(pattern, "", sentence)
                        
            #elimino eventuali spazi supplementari all'inizio e alla fine della frase
            sentence = sentence.strip()
            
            #controllo che la frase non cominci con un segno di interpunzione
            punteggiatura = ['-','.',',',';',':','!','?']
                        
            for p in punteggiatura:
                if sentence.startswith(p):
                    sentence = sentence[1:]
                    sentence = sentence.strip()
                    
            #elimino eventuali spazi supplementari all'inizio e alla fine della frase
            sentence = sentence.strip()

            if sentence:
                if len(sentence.split()) > 1:
                    dataset_new.append(sentence)
        
    return dataset_new

In [10]:
def save_dataframe(file_name, dataframe_to_save):
    
    try:
        dataframe_to_save.to_csv(path_dataset+file_name+".csv", index = False, sep = "\t")
        print("File salvato correttamente")
        
    except:
        print("Errore nel salvataggio")
    
    return 

In [11]:
#frase = " [Musica], hai ragione, [sai] che [Roberto] diceva: (Musica del basso) (Parla) Prima [di] tutto ho inventato [me] stesso [musica], l'universo è stato mischiato, le carte in regola sono state girate (fine musica)."
#frase = " [Roberto] [Musica] [Applausi]  Grazie [davvero] e buon pomeriggio a tutti."
#print("La frase è: ", frase)
#print()
#prova = clean_sentences([frase])

In [12]:
#prova

In [13]:
#for i in dataset:
#    if "Musica" in i:
#       print(i)

# Data analysis

In [14]:
with open("/Users/eleonoracocciu/Desktop/TESI_magistrale/TEDx/mTEDx_it/data/train/txt/train_txt.txt", 'r') as file:
    train_data = file.readlines()

In [15]:
with open("/Users/eleonoracocciu/Desktop/TESI_magistrale/TEDx/mTEDx_it/data/valid/txt/valid_txt.txt", 'r') as file:
    validation_data = file.readlines()

In [16]:
with open("/Users/eleonoracocciu/Desktop/TESI_magistrale/TEDx/mTEDx_it/data/test/txt/test_txt.txt", 'r') as file:
    test_data = file.readlines()

In [17]:
get_statistics([train_data, validation_data, test_data], ["train_data", "validation_data", "test_data"])

train_data       : 49964 sentences
validation_data  : 931 sentences
test_data        : 999 sentences

Totale           : 51894 sentences


In [18]:
train_data.extend(validation_data)
train_data.extend(test_data)

In [19]:
print("{0:17}: {1} sentences".format("Dimensione del dataset", len(train_data)))

Dimensione del dataset: 51894 sentences


In [20]:
dataset = train_data

In [21]:
print("{0:17}: {1} sentences".format("Dimensione del dataset", len(dataset)))

Dimensione del dataset: 51894 sentences


In [22]:
flusso_dataset_completo_unificato = pd.DataFrame(dataset, columns=['Original_sentences'])
save_dataframe("flusso_TEDx_completo_unificato", flusso_dataset_completo_unificato)

File salvato correttamente


In [23]:
dataset[:20]

["C'è una cosa che io faccio sempre, quando prendo un aereo, grande o piccolo come questo.\n",
 "All'atterraggio e al decollo io guardo fuori dal finestrino.\n",
 'Probabilmente lo fate anche voi, anche quando si decolla dalla propria città che si conosce benissimo, o si atterra in un posto dove si è arrivati già tante volte.\n',
 'Perché in realtà, una delle cose che ci piace del volare, è che ci consente di vedere le cose con una prospettiva nuova.\n',
 'Gli astronauti quando sono nello spazio per la prima volta vedono la terra da lassù, provano una sensazione così unica che gli è stato dato un nome: "Overview Effect".\n',
 'È quando si rendono conto, guardando la Terra, che ci sono confini, fisici e mentali, che sono solo costruzioni artificiali, perché noi abbiamo due occhi, solo davanti, e due piedi, solo per terra, e ci abituiamo a pensare a tutto con questa prospettiva unica, delle cose che si svolgono davanti a noi.\n',
 'Non è così.\n',
 'Da bambini, neonati, ci rotoliamo di p

# Data cleaning

In [24]:
square_list = []
round_list = []

for i in dataset:
    r1 = re.findall(r'\[.*?\]', i)
    r2 = re.findall(r'\(.*?\)', i)
    square_list.extend(r1)
    round_list.extend(r2)
    
square_list = list(set(square_list))
round_list = list(set(round_list))

In [25]:
print("{0:19}: {1}".format("Elementi nelle quadre", len(square_list)))
print("{0:21}: {1}".format("Elementi nelle tonde", len(round_list)))

Elementi nelle quadre: 362
Elementi nelle tonde : 201


In [26]:
square_list

['[valido]',
 '[1- Alimentazione forzata per il foie gras]',
 '[A.DE.]',
 '[20 secondi dopo]',
 '[- La violenza si ferma solo con altra violenza]',
 '[come questa]',
 '[Pensate a]',
 '[RG]',
 "[Scoprite il più grande green murale d'Europa]",
 '[Musica]',
 '[muscoloso]',
 "[Tesoro, significa che sei un po' nero, ecco.]",
 '[Il design ha bisogno del cibo]',
 '[celebriamo come]',
 '[l’operazione]',
 '[In dialetto]',
 "[suona l'<i>Almanacco del giorno dopo</i>]",
 '[Rivoluzione Tecnologica]',
 '[di essere]',
 '[che accade]',
 '[6 – Esperimenti crudeli]',
 '[che gli uomini non solo possono,]',
 '[7 – Ammassati durante il trasporto]',
 '[Sì!]',
 '[per]',
 '[Ci siamo riusciti]',
 "[l'enigma]",
 '[Diventi responsabile, per sempre, di ciò che hai addomesticato.]',
 '[dei casi]',
 '[modello]',
 '[perché ho i risultati del mio test del DNA]',
 '[3 – Triturati vivi]',
 '[solo economico]',
 '[e]',
 '[Ti dono le stesse opportunità e capacità degli uomini]',
 '[Un altro caso di violenza sulle donne!]

In [27]:
round_list

['(parti per milione)',
 '(La musica finisce)',
 '(Sussurra)',
 '(AUDIO)',
 '(Rumore di vetri rotti)',
 '(questa è la stessa foto un secolo dopo)',
 '(Musica in sottofondo)',
 '(La musica si interrompe)',
 '(tranne quella rubata ai quartieri vicini)',
 '(quando dovevano inventare la lampadina)',
 '("Questi marocchini in dialetto bolognese)',
 '(suono di goccia)',
 '(Parla)',
 '(Portoghese)',
 '(Asse X)',
 '(Pianoforte)',
 '(Pubblico: 25)',
 '(Nella vignetta Charlie Brown dice: Mi sembra che ci sia qualcun altro che sta imparando dai miei errori al posto mio)',
 '(EN)',
 '(Risate e applausi)',
 '(Grazie)',
 '(Sorriso e Risatine)',
 "(Video: e vediamo se effettivamente l'acclimatamento...)",
 '(finte parole inglesi)',
 '(NIDA)',
 '(Suono di campanacci)',
 '(Respiri)',
 '(Insieme)',
 '(suono di gocce)',
 '(Suoni ambientali)',
 '(Voci)',
 '(-18)',
 '(faccio la versione vegetariana delle lasagne della nonna)',
 '(Video: saliamo se riusciamo fino al campo 2)',
 '(Risate.)',
 '(Fine musica)',

In [28]:
clean_sentences(["?” Lucas: “contrare” Io: “contrare?” Lucas: “Poi c’era nonno” Io: “Non ho capito” Lucas: “ncontrare, incontrare, poi c’era nonno… c’era nonno” Io: “C'era nonno prima” Lucas: “Si” Io: “E ora nonno?” Lucas: “Non c'è più” Io: “E come sta Lucas?"])

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 218.24it/s]


["' Lucas: 'contrare' Io: 'contrare? ' Lucas: 'Poi c'era nonno' Io: 'Non ho capito' Lucas: 'ncontrare, incontrare, poi c'era nonno… c'era nonno' Io: 'C'era nonno prima' Lucas: 'Si' Io: 'E ora nonno? ' Lucas: 'Non c'è più' Io: 'E come sta Lucas?"]

## Dataset cleaning

In [29]:
list_of_sentences = clean_sentences(dataset)

100%|███████████████████████████████████| 51894/51894 [00:15<00:00, 3308.02it/s]


In [30]:
print("{0:17}: {1} sentences".format("Dimensione del dataset", len(list_of_sentences)))

Dimensione del dataset: 50094 sentences


Controllo che non siano rimaste parentesi quadre e tonde nelle frasi

In [31]:
for frase in list_of_sentences:
    if '(' in frase or ')' in frase or '[' in frase or ']' in frase:
        print(frase)

Controllo che non siano presenti frasi che cominciano con un segno di interpunzione

In [32]:
punteggiatura = ['-','.',',',';',':','!','?']

for i, frase in enumerate(list_of_sentences):    
    for p in punteggiatura:
        if frase.startswith(p):
            print(i, frase)

342 ..e sono tutte presenti, quindi è un momento storico.
18580 . potete tornare comodi, grazie!
48801 - Oh, Fede, ma ho rotto la pelle; - Oh, su, su Fede, che è ora di montare.


In [33]:
for frase in dataset:
    if "Quel vascello valeva" in frase:
        print(frase)

Quel vascello valeva €400.000.



In [34]:
for frase in list_of_sentences:
    if frase[-1] not in ['.', ';', '?', '!', '"']:
        print(frase)

"...de l'ombre e de la pioggia, a passi lenti, toccando un poco la vita futura; per ch'io dissi: 'Maestro, esti tormenti crescerann'ei dopo la gran sentenza, o fier minori, o saran sì cocenti? '
E chi son quei che 'l fenno? '
Esisterà al mondo una barca che mi permetta di far questo? '
Ecco, questo... Grazie
Grazie Applausi
In questa frase, l'autore di questa mail dice che 'the vast preponderance of scientific report is set against your opinion, you are a distinct minority.'
Voi siete in netta minoranza.'
'Estote parate'
Ecco, è possibile raggiungere tutti, perché esiste questo grande catalizzatore che è il desiderio, e noi è questo che dobbiamo accendere, che dobbiamo incrociare ed è la grande rivoluzione che deve accadere a scuola e può accadere senza nessuna legge, perché da domani mattina io posso andare a scuola, con questo desiderio di scoprire il vero, il bene e il bello che c'è dentro la realtà, insieme ai miei alunni, insieme ai miei colleghi, e questo è semplice, è semplice c

In [35]:
list_of_sentences[:10]

["C'è una cosa che io faccio sempre, quando prendo un aereo, grande o piccolo come questo.",
 "All'atterraggio e al decollo io guardo fuori dal finestrino.",
 'Probabilmente lo fate anche voi, anche quando si decolla dalla propria città che si conosce benissimo, o si atterra in un posto dove si è arrivati già tante volte.',
 'Perché in realtà, una delle cose che ci piace del volare, è che ci consente di vedere le cose con una prospettiva nuova.',
 'Gli astronauti quando sono nello spazio per la prima volta vedono la terra da lassù, provano una sensazione così unica che gli è stato dato un nome: "Overview Effect".',
 'È quando si rendono conto, guardando la Terra, che ci sono confini, fisici e mentali, che sono solo costruzioni artificiali, perché noi abbiamo due occhi, solo davanti, e due piedi, solo per terra, e ci abituiamo a pensare a tutto con questa prospettiva unica, delle cose che si svolgono davanti a noi.',
 'Non è così.',
 'Da bambini, neonati, ci rotoliamo di più, i neonati 

**Salvo la lista di frasi ripulite in un dataframe e poi salvo il dataframe**

In [36]:
flusso_dataset_completo_ripulito = pd.DataFrame(list_of_sentences, columns=['Sentences'])

In [37]:
flusso_dataset_completo_ripulito

,Sentences
0,"C'è una cosa che io faccio sempre, quando pren..."
1,All'atterraggio e al decollo io guardo fuori d...
2,"Probabilmente lo fate anche voi, anche quando ..."
3,"Perché in realtà, una delle cose che ci piace ..."
4,Gli astronauti quando sono nello spazio per la...
...,...
50089,"Allora, essere ottimisti forse oggi è semplice..."
50090,E una persona che ha questa capacità non può n...
50091,E allora come faccio io per cambiare il mondo?
50092,"Ci sono molti che vogliono cambiare il mondo, ..."


In [38]:
save_dataframe("flusso_TEDx_completo_ripulito", flusso_dataset_completo_ripulito)

File salvato correttamente


# Training set extraction

**Ricaviamo dei training set a complessità crescente**

In [39]:
def create_df(list_of_sentences, dist, perc):
    
    list_of_lists = []
    
    for i, el in enumerate(list_of_sentences):
        
        if i < len(list_of_sentences) - dist:
            
            #la frase 2 segue la frase 1 --> Classe 0
            list_of_lists.append([list_of_sentences[i], list_of_sentences[i+1], '0'])
            
            #la frase 2 non segue la frase 1 --> Classe 1
            list_of_lists.append([list_of_sentences[i], list_of_sentences[i+dist], '1'])
    
    np_array = np.array(list_of_lists, dtype=object)
    
    df = pd.DataFrame(np_array, columns=['Sentence_1', 'Sentence_2', "Class"])
    
    #calcolo le quantità di esempi positivi e negativi
    diz_count_classes = df['Class'].value_counts().to_dict()
    
    #prendo la quantità di esempi positivi (frasi che non sono consecutive)
    qty_positive_class = diz_count_classes['1']
    
    #calcolo la quantità di frasi da invertire a seconda del parametro della percentuale e arrotondo per eccesso
    #al numero intero più vicino
    sentences_to_invert = math.ceil(qty_positive_class / 100 * perc)
    
    #Costruisco la lista degli indici degli esempi, isolandoli
    idx_positive_examples = list(df[df['Class'] == '1'].index)
    
    #eseguo l'inversione delle frasi
    for idx in idx_positive_examples[:sentences_to_invert]:
        tmp = df.loc[idx, 'Sentence_1']
        df.loc[idx, 'Sentence_1'] = df.loc[idx, 'Sentence_2']
        df.loc[idx, 'Sentence_2'] = tmp
                
    #ordino il dataset in base alla colonna della Classe, in modo decrescente
    df.sort_values(by=['Class'], inplace = True, ascending = False)
    
    #resetto l'indice
    df.reset_index(inplace = True, drop = True)
    
    display(df)
    print()
    print("Caratteristiche del dataset con esempi positivi a distanza {}".format(dist))
    print()
    print("Righe: {} \t Colonne: {}".format(df.shape[0], df.shape[1]))
    print()
    print("Dettaglio delle classi")
    print(df['Class'].value_counts())
    print()
    print("Percentuale di esempi positivi da invertire: {}%".format(perc))
    print("Sono stati invertiti {} esempi positivi".format(sentences_to_invert))
    print()
    print()
    
    return df

In [40]:
df_dist_5 = create_df(list_of_sentences, 5, 30)
print("-------------------------------------------------------------")
save_dataframe('df_TEDx_dist_5', df_dist_5)

,Sentence_1,Sentence_2,Class
0,Quante volte sentite quello dall'altra parte c...,Non devi calpestare B per andare da A a C.,1
1,Le forbici molecolari hanno tagliato la doppia...,Tant'è che un RNA guida costa poche decine di ...,1
2,È semplicissimo.,Abbiamo fatto il taglio.,1
3,Dovremmo semplicemente cambiare la sequenza de...,E chi ricuce?,1
4,È semplice da disegnare e poi è anche material...,Ci sono vari meccanismi che la cellula utilizz...,1
...,...,...,...
100173,"Avevo incontrato un muro, ci avevo sbattuto co...",Diagnosi di glomerulonefrite cronica da immuno...,0
100174,"In teatro, se non ci sei, se non fai quello ch...","In carcere potrebbero fartela pagare, ma nel f...",0
100175,"In carcere potrebbero fartela pagare, ma nel f...",Se il tuo compagno dietro le quinte non si ric...,0
100176,Era come se tutto si fosse fermato.,"Avevo incontrato un muro, ci avevo sbattuto co...",0



Caratteristiche del dataset con esempi positivi a distanza 5

Righe: 100178 	 Colonne: 3

Dettaglio delle classi
1    50089
0    50089
Name: Class, dtype: int64

Percentuale di esempi positivi da invertire: 30%
Sono stati invertiti 15027 esempi positivi


-------------------------------------------------------------
File salvato correttamente


In [41]:
df_dist_10 = create_df(list_of_sentences, 10, 30)
print("-------------------------------------------------------------")
save_dataframe('df_TEDx_dist_10', df_dist_10)

,Sentence_1,Sentence_2,Class
0,"E le cose positive, quanto tarderemmo a fare u...",Io credo che se si vuole veramente cambiare il...,1
1,"È sotterraneo, è subcutaneo.",La velocità repentina di poter ottenere una co...,1
2,"E questo ci porta al terzo punto, da dividere ...",La fatica di poterla raggiungere.,1
3,Quel giorno avevamo veramente salvato le balene.,Decido di navigare attorno a questo piccolo ar...,1
4,Una è il disagio.,La fatica impiegata per ottenerla.,1
...,...,...,...
100163,"La felicità è un bisogno serio di tutti, di tu...",Quello che noi cerchiamo di fare nel carcere d...,0
100164,Quello che noi cerchiamo di fare nel carcere d...,Alcuni di loro questi strumenti non li hanno m...,0
100165,Alcuni di loro questi strumenti non li hanno m...,"Nel carcere di Bologna si può studiare, si può...",0
100166,"Nel carcere di Bologna si può studiare, si può...",Ci sono persone che hanno preso più di una lau...,0



Caratteristiche del dataset con esempi positivi a distanza 10

Righe: 100168 	 Colonne: 3

Dettaglio delle classi
1    50084
0    50084
Name: Class, dtype: int64

Percentuale di esempi positivi da invertire: 30%
Sono stati invertiti 15026 esempi positivi


-------------------------------------------------------------
File salvato correttamente


In [42]:
df_dist_20 = create_df(list_of_sentences, 20, 30)
print("-------------------------------------------------------------")
save_dataframe('df_TEDx_dist_20', df_dist_20)

,Sentence_1,Sentence_2,Class
0,E cosa insegniamo ai nostri figli?,Io credo che se si vuole veramente cambiare il...,1
1,"In Sri Lanka, per esempio, si passò da 3 milio...",Di nuovo assistiamo pericolosamente ai fatti c...,1
2,"""Wahaha"" La vita è una questione di scelte, e ...",Il venditore afgano ha visto esploder gambe An...,1
3,"Paesaggio mobile, che si dissolveva pochi ista...",Con i Woodaabe ho condiviso le piogge sacre de...,1
4,E la National Academy of Science ha stimato ch...,Ma come fare quindi a distinguere un fatto da ...,1
...,...,...,...
100143,"Sono sue le sceneggiature di ""The Heat - La Sf...","Anche Edward, a un certo punto, esce dal carce...",0
100144,"Anche Edward, a un certo punto, esce dal carce...",Forse cominciate a capire perché bisogna inter...,0
100145,Forse cominciate a capire perché bisogna inter...,Abbott ci dice una cosa: Attenzione!,0
100146,Abbott ci dice una cosa: Attenzione!,"La felicità è un bisogno serio di tutti, di tu...",0



Caratteristiche del dataset con esempi positivi a distanza 20

Righe: 100148 	 Colonne: 3

Dettaglio delle classi
1    50074
0    50074
Name: Class, dtype: int64

Percentuale di esempi positivi da invertire: 30%
Sono stati invertiti 15023 esempi positivi


-------------------------------------------------------------
File salvato correttamente


In [43]:
df_dist_30 = create_df(list_of_sentences, 30, 30)
print("-------------------------------------------------------------")
save_dataframe('df_TEDx_dist_30', df_dist_30)

,Sentence_1,Sentence_2,Class
0,"Ci sono aspetti paradossali, magari possiamo p...",Io credo che se si vuole veramente cambiare il...,1
1,"È una conoscenza un po' differente, dove si ve...",Vedevamo persone soltanto in funzione di aggiu...,1
2,Quali sono questi smarrimenti?,"Ve lo ho detto, tra me e voi c'è quello che vi...",1
3,"""Io non mi muover˜ò.","E ci˜ò che un tempo era una caccia, sicurament...",1
4,Alcuni sono abbastanza evidenti.,"Io sono analogico, voi siete digitali.",1
...,...,...,...
100123,"Ha un carattere violento, irascibile e commett...",L'unica cosa che può fare nei periodi in cui n...,0
100124,L'unica cosa che può fare nei periodi in cui n...,"Per cui torna in carcere, perde tutta la sua r...",0
100125,"Per cui torna in carcere, perde tutta la sua r...","Edward Banker, nasce anch'egli negli Stati Uni...",0
100126,"Edward Banker, nasce anch'egli negli Stati Uni...",Questa macchina da scrivere cambierà la vita d...,0



Caratteristiche del dataset con esempi positivi a distanza 30

Righe: 100128 	 Colonne: 3

Dettaglio delle classi
1    50064
0    50064
Name: Class, dtype: int64

Percentuale di esempi positivi da invertire: 30%
Sono stati invertiti 15020 esempi positivi


-------------------------------------------------------------
File salvato correttamente


In [44]:
df_dist_5

,Sentence_1,Sentence_2,Class
0,Quante volte sentite quello dall'altra parte c...,Non devi calpestare B per andare da A a C.,1
1,Le forbici molecolari hanno tagliato la doppia...,Tant'è che un RNA guida costa poche decine di ...,1
2,È semplicissimo.,Abbiamo fatto il taglio.,1
3,Dovremmo semplicemente cambiare la sequenza de...,E chi ricuce?,1
4,È semplice da disegnare e poi è anche material...,Ci sono vari meccanismi che la cellula utilizz...,1
...,...,...,...
100173,"Avevo incontrato un muro, ci avevo sbattuto co...",Diagnosi di glomerulonefrite cronica da immuno...,0
100174,"In teatro, se non ci sei, se non fai quello ch...","In carcere potrebbero fartela pagare, ma nel f...",0
100175,"In carcere potrebbero fartela pagare, ma nel f...",Se il tuo compagno dietro le quinte non si ric...,0
100176,Era come se tutto si fosse fermato.,"Avevo incontrato un muro, ci avevo sbattuto co...",0


In [45]:
df_dist_5[df_dist_5['Class'] == '0']

,Sentence_1,Sentence_2,Class
50089,"Lo vedete qui alle mie spalle, un aeroplano di...","Un'idea grandiosa, potremmo dire, il primo jum...",0
50090,Che tanti altri partecipino a quello che hai s...,Siamo a Mantova.,0
50091,E siamo anche degli adulti che hanno speriment...,E questo ha determinato un riversamento di noi...,0
50092,L'aspetto diciamo critico di questi finanziame...,Cioè sono molto difficili da riuscire a prendere.,0
50093,E questo mi fa sovvenire una domanda.,"Non so se sapete perché si dice: ""Me la faccio...",0
...,...,...,...
100173,"Avevo incontrato un muro, ci avevo sbattuto co...",Diagnosi di glomerulonefrite cronica da immuno...,0
100174,"In teatro, se non ci sei, se non fai quello ch...","In carcere potrebbero fartela pagare, ma nel f...",0
100175,"In carcere potrebbero fartela pagare, ma nel f...",Se il tuo compagno dietro le quinte non si ric...,0
100176,Era come se tutto si fosse fermato.,"Avevo incontrato un muro, ci avevo sbattuto co...",0


In [46]:
df_dist_5[df_dist_5['Class'] == '1']

,Sentence_1,Sentence_2,Class
0,Quante volte sentite quello dall'altra parte c...,Non devi calpestare B per andare da A a C.,1
1,Le forbici molecolari hanno tagliato la doppia...,Tant'è che un RNA guida costa poche decine di ...,1
2,È semplicissimo.,Abbiamo fatto il taglio.,1
3,Dovremmo semplicemente cambiare la sequenza de...,E chi ricuce?,1
4,È semplice da disegnare e poi è anche material...,Ci sono vari meccanismi che la cellula utilizz...,1
...,...,...,...
50084,Voglio tornare dove la mia vita è cambiata.,"Dove passo io, una mamma con un passeggino ha ...",1
50085,E dove si trova la tua mente in questo momento?,Fate apparire un lieve sorriso sulle vostre la...,1
50086,"A Parigi stavamo sempre insieme, mi portavi ai...",Ti devo interrompere.,1
50087,"Grazie a Dio, grazie a Dio, grazie a Dio no.","Io se fossi pazzo, non mi farei più rintracciare.",1


In [47]:
import random

def assign_ids(df):
    
    df_new = df.copy()
    
    li = range(1,200000)
    
    columns_of_ids = random.sample(li, k=df.shape[0])
    
    df_new.insert(0, 'Id', columns_of_ids)
    
    print("Presenza di id duplicati: {}".format("Sì" if df_new['Id'].duplicated().any() else "No"))
    
    return df_new

In [48]:
#df_dist_5 = pd.read_csv("df_TEDx_dist_5.csv", sep = "\t")
#df_dist_10 = pd.read_csv("df_TEDx_dist_10.csv", sep = "\t")
#df_dist_20 = pd.read_csv("df_TEDx_dist_20.csv", sep = "\t")
#df_dist_30 = pd.read_csv("df_TEDx_dist_30.csv", sep = "\t")

In [49]:
df_dist_5_with_ids = assign_ids(df_dist_5)
df_dist_10_with_ids = assign_ids(df_dist_10)
df_dist_20_with_ids = assign_ids(df_dist_20)
df_dist_30_with_ids = assign_ids(df_dist_30)

Presenza di id duplicati: No
Presenza di id duplicati: No
Presenza di id duplicati: No
Presenza di id duplicati: No


In [50]:
df_dist_5_with_ids

,Id,Sentence_1,Sentence_2,Class
0,83300,Quante volte sentite quello dall'altra parte c...,Non devi calpestare B per andare da A a C.,1
1,83146,Le forbici molecolari hanno tagliato la doppia...,Tant'è che un RNA guida costa poche decine di ...,1
2,23967,È semplicissimo.,Abbiamo fatto il taglio.,1
3,138269,Dovremmo semplicemente cambiare la sequenza de...,E chi ricuce?,1
4,179316,È semplice da disegnare e poi è anche material...,Ci sono vari meccanismi che la cellula utilizz...,1
...,...,...,...,...
100173,50147,"Avevo incontrato un muro, ci avevo sbattuto co...",Diagnosi di glomerulonefrite cronica da immuno...,0
100174,181833,"In teatro, se non ci sei, se non fai quello ch...","In carcere potrebbero fartela pagare, ma nel f...",0
100175,44121,"In carcere potrebbero fartela pagare, ma nel f...",Se il tuo compagno dietro le quinte non si ric...,0
100176,34673,Era come se tutto si fosse fermato.,"Avevo incontrato un muro, ci avevo sbattuto co...",0


In [51]:
save_dataframe('df_TEDx_dist_5_with_ids', df_dist_5_with_ids)

File salvato correttamente


In [52]:
save_dataframe('df_TEDx_dist_10_with_ids', df_dist_10_with_ids)

File salvato correttamente


In [53]:
save_dataframe('df_TEDx_dist_20_with_ids', df_dist_20_with_ids)

File salvato correttamente


In [54]:
save_dataframe('df_TEDx_dist_30_with_ids', df_dist_30_with_ids)

File salvato correttamente


In [56]:
for row in df_dist_5_with_ids.itertuples():
    if "La prima componente fondamentale è Cas9." in row.Sentence_1 \
    or "La prima componente fondamentale è Cas9." in row.Sentence_2:
        print(row)

Pandas(Index=39, Id=26761, Sentence_1='La prima componente fondamentale è Cas9.', Sentence_2='È come dire che è un manuale di istruzioni da 3 miliardi di caratteri.', Class='1')
Pandas(Index=129, Id=69880, Sentence_1="Dov'è la potenza dirompente di CRISPS - Cas?", Sentence_2='La prima componente fondamentale è Cas9.', Class='1')
Pandas(Index=93198, Id=40636, Sentence_1='Funziona un po\' come un kit di "taglia e cuci" per riscrivere il DNA.', Sentence_2='La prima componente fondamentale è Cas9.', Class='0')
Pandas(Index=93199, Id=103205, Sentence_1='La prima componente fondamentale è Cas9.', Sentence_2='Una proteina, specificamente un enzima, che taglia il DNA.', Class='0')


In [57]:
for i, el in enumerate(dataset):
    if "La prima componente fondamentale è Cas9." in el:
        print(i, el)

28756 La prima componente fondamentale è Cas9.



In [65]:
for f in list_of_sentences:
    if ';' in f:
        print(f)
        print()

Ero direttore di un giornale, venivo da anni molto intensi; sedici ore al giorno di lavoro, sempre attaccata al cellulare, sempre a rincorrere quelle che mi sembravano costanti emergenze.

La nostra esperienza è iniziata nel 2015 quando il professore di impianti e sistemi aerospaziali di Padova, Alessandro Francesconi, ci ha suggerito di provare uno di quei bandi che ogni anno l'Agenzia Spaziale Europea mette a disposizione per gli studenti universitari per avvicinarli già dai primi anni alla ricerca in campo spaziale; quindi, potete immaginare il nostro entusiasmo di fronte a questa proposta.

E l'abbiamo poi adattata allo spazio; cosa che magari, ad una prima impressione, può sembrare un po' banale, ma in realtà quello che il nostro team si è trovato ad affrontare è stata una vera e propria sfida nei confronti di anni di studio.

Per cui è vero che gli ingegneri, quella gente lì, sono importanti, hanno un lavoro serio; ma anche i sognatori, anche quelli che inventano cose, pur non fa

E sto parlando di una situazione quotidiana standard; ma pensate ad un evento imprevisto, un piccolo incidente.

Ok, la morale è semplice: con una serie di sì io forse vado da qualche parte; con dei "No" non inizio nemmeno.

Si può lavorare in contesti di degrado urbano; si può lavorare per valorizzare dei beni culturali; si possono utilizzare tecniche per l'integrazione di gruppi di etnie diverse; si può lavorare nelle scuole come didattica alternativa; si può lavorare con gruppi di qualsiasi tipo: di manager, di attori, di sportivi, di scout.

Una di loro aveva un passamontagna, e un enorme maglione nero strappato; l'altra, un cappellino con la tesa e un bomber grigio scuro.

Sarei potuto rientrare in Italia a raccontare tutto, a denunciare tutto ciò che avevo visto; ma sapevo che non sarebbe servito a niente: altri, prima di me, lo avevano già fatto.

Dovevo restare; volevo restare, con tutto me stesso.

Si tramanda di generazione in generazione, per consuetudine; e difficilmente ci

In [55]:
for f in dataset:
    if ';' in f:
        print(f)
        print()

Ero direttore di un giornale, venivo da anni molto intensi; sedici ore al giorno di lavoro, sempre attaccata al cellulare, sempre a rincorrere quelle che mi sembravano costanti emergenze.


La nostra esperienza è iniziata nel 2015 quando il professore di impianti e sistemi aerospaziali di Padova, Alessandro Francesconi, ci ha suggerito di provare uno di quei bandi che ogni anno l'Agenzia Spaziale Europea mette a disposizione per gli studenti universitari per avvicinarli già dai primi anni alla ricerca in campo spaziale; quindi, potete immaginare il nostro entusiasmo di fronte a questa proposta.


E l'abbiamo poi adattata allo spazio; cosa che magari, ad una prima impressione, può sembrare un po' banale, ma in realtà quello che il nostro team si è trovato ad affrontare è stata una vera e propria sfida nei confronti di anni di studio.


Per cui è vero che gli ingegneri, quella gente lì, sono importanti, hanno un lavoro serio; ma anche i sognatori, anche quelli che inventano cose, pur non



Allora, lui comincia a intraprendere quella strada, arriva nel bosco, il bosco gli si apre: è un bosco odoroso, meraviglioso; ma soprattutto vede una luce al di là del bosco e viene accolto in una casa enorme, bellissima, con una principessa che lo fa entrare e gli regala pietre preziose, diamanti, gemme.


(Risate) Adesso magari vogliamo ridurre il tono di grassi saturi, dobbiamo cercare di fare una pasta frolla non con il burro ma magari con olio; è molto difficile, poi ve lo dirò.


Come neurologo lo dico perché, dirigendo il centro coma, io questo vedo: che molte malattie è molto difficile curarle; però posso lavorare sull'altra componente che determina la disabilità, l'ambiente.


Se hai degli amici, stai meglio; ma non stai meglio soltanto in termini di benessere: stai proprio meglio!


Seppellite i vostri morti, mettete al sicuro le vostre famiglie; e poi venite qua.


Faremo i soldi, voi ricostruite le vostre case; però state con me, state insieme.


Ognuno spera sempre di es



Diventa difficile, dicevamo; ma Leo ha incominciato, un giorno, a fare la pizza con la sua mamma, a impastare la pizza.


Adesso alcuni di voi hanno sorriso perché hanno capito al volo; noi ci abbiamo messo due settimane, a capire.


Alessandro; autistico; interpretazione letterale.


C'è una ruota che gira sempre: i ragazzi appoggiano la loro pizza su questa ruota, la ruota porta l'impasto dentro il forno; il forno la cuoce; la stessa ruota lo porta fuori, e l'altro Cameriere autistico lo prende e lo porta in sala.


Aspettativa di vita; 40 anni; reddito medio equivalente a quello di oggi: 2500 euro.


Noi vediamo cosa sta succedendo agli animali, agli Albatros nelle Midway; ma in realtà la stessa cosa sta succedendo a noi come esseri umani.


Intendo un modello di business che è guidato da un unico obiettivo: guardare dove sia il valore, raccoglierlo e concentrarlo; estrarlo e concentrarlo; estrarlo e concentrarlo.


Complessivamente, in Europa, quando è nata Nativa eravamo il pazi